In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lllllllll/project.csv
/kaggle/input/haxckathon/training_set_features.csv
/kaggle/input/haxckathon/test_set_features.csv
/kaggle/input/haxckathon/training_set_labels.csv


In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Load the training dataset
train_data = pd.read_csv('/kaggle/input/haxckathon/training_set_features.csv')
train_labels = pd.read_csv('/kaggle/input/haxckathon/training_set_labels.csv')

# Separate features and target variables
n =  pd.read_csv('/kaggle/input/haxckathon/training_set_labels.csv')
train_data[['xyz_vaccine', 'seasonal_vaccine']] = n[['xyz_vaccine', 'seasonal_vaccine']]

# Recompute numeric and categorical columns after dropping target columns
numeric_columns = train_data.select_dtypes(include=['number']).columns
categorical_columns = train_data.select_dtypes(include=['object']).columns

# Fill missing values with median for numeric columns
train_data[numeric_columns] = train_data[numeric_columns].fillna(train_data[numeric_columns].median())

# Fill missing values with mode for categorical columns
for column in categorical_columns:
    train_data[column] = train_data[column].fillna(train_data[column].mode()[0])

# One-hot encode categorical variables
train_data = pd.get_dummies(train_data, columns=categorical_columns)

# Separate features and target variables again after encoding
X_train = train_data.drop(columns=['xyz_vaccine', 'seasonal_vaccine'])
y_train_xyz = train_data['xyz_vaccine'].astype(int)
y_train_seasonal = train_data['seasonal_vaccine'].astype(int)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train Logistic Regression model for xyz_vaccine
log_reg_xyz = LogisticRegression(max_iter=1000)
log_reg_xyz.fit(X_train_scaled, y_train_xyz)

# Train Logistic Regression model for seasonal_vaccine
log_reg_seasonal = LogisticRegression(max_iter=1000)
log_reg_seasonal.fit(X_train_scaled, y_train_seasonal)

# Load the test dataset
test = pd.read_csv('/kaggle/input/haxckathon/test_set_features.csv')
test_data = test.drop(columns='respondent_id')

# Recompute numeric and categorical columns for the test data
numeric_columns_test = test_data.select_dtypes(include=['number']).columns
categorical_columns_test = test_data.select_dtypes(include=['object']).columns

# Preprocess the test data (same steps as training data)
test_data[numeric_columns_test] = test_data[numeric_columns_test].fillna(train_data[numeric_columns].median())

for column in categorical_columns_test:
    if column in train_data.columns:
        test_data[column] = test_data[column].fillna(train_data[column].mode()[0])
    else:
        test_data[column] = test_data[column].fillna('missing')

test_data = pd.get_dummies(test_data, columns=categorical_columns_test)

# Ensure the test data has the same columns as the training data
missing_cols = set(X_train.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X_train.columns]

# Standardize the features
test_data_scaled = scaler.transform(test_data)

# Predict probabilities for xyz_vaccine
test_pred_proba_xyz = log_reg_xyz.predict_proba(test_data_scaled)[:, 1]

# Predict probabilities for seasonal_vaccine
test_pred_proba_seasonal = log_reg_seasonal.predict_proba(test_data_scaled)[:, 1]

# Calculate ROC AUC scores
roc_auc_xyz = roc_auc_score(train_labels['xyz_vaccine'], log_reg_xyz.predict_proba(X_train_scaled)[:, 1])
roc_auc_seasonal = roc_auc_score(train_labels['seasonal_vaccine'], log_reg_seasonal.predict_proba(X_train_scaled)[:, 1])

print(f'ROC AUC Score for XYZ Vaccine: {roc_auc_xyz}')
print(f'ROC AUC Score for Seasonal Vaccine: {roc_auc_seasonal}')



# Prepare submission dataframe
submission_df = pd.DataFrame({
    'respondent_id': test['respondent_id'],
    'xyz_vaccine': test_pred_proba_xyz,
    'seasonal_vaccine': test_pred_proba_seasonal
})

# Save to CSV
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


ROC AUC Score for XYZ Vaccine: 0.8379214538424193
ROC AUC Score for Seasonal Vaccine: 0.8559730328650688
